In [1]:
import pandas as pa
import yfinance as yf
from datetime import datetime
import plotly.express as plexp

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
yf.pdr_override()
start = datetime.now() - pa.DateOffset(months=3)
end = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG', 'IBM']

In [3]:
df_list = []
for ticker in tickers:
    data = yf.download(ticker, start=start, end=end)
    df_list.append(data)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
df = pa.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())
print(df.tail())
df = df.reset_index()
print(df.head())
print(df.tail())

                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-11-28  189.779999  191.080002  189.399994  190.399994  190.157349   
       2023-11-29  190.899994  192.089996  188.970001  189.369995  189.128662   
       2023-11-30  189.839996  190.320007  188.190002  189.949997  189.707916   
       2023-12-01  190.330002  191.559998  189.229996  191.240005  190.996292   
       2023-12-04  189.979996  190.050003  187.449997  189.429993  189.188583   

                     Volume  
Ticker Date                  
AAPL   2023-11-28  38415400  
       2023-11-29  43014200  
       2023-11-30  48794400  
       2023-12-01  45679300  
       2023-12-04  43389500  
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
IBM    2024-02-22  182.449997  184.550003  181.929993  184.2

In [6]:
img_1 = plexp.line(
    df,
    x= 'Date',
    y= 'Close',
    color= 'Ticker',
    title= 'Stock Prices',
)
img_1.show()

In [7]:
img_2 = plexp.area(df,
                   x = 'Date',
                   y = 'Close',
                   facet_col= 'Ticker',
                   labels= {'Date':'Date', 'Close': 'Close Price', 'Ticker': 'Stock Ticker'},
                   title= 'Stock Price Movements'
                   )
img_2.show()

In [8]:
df['MAverage10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MAverage20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)


In [9]:
for ticker, group in df.groupby('Ticker'):
    print(f"Moving Average for {ticker}")
    print(group[['MAverage10', 'MAverage20']])
    print("\n\n")

Moving Average for AAPL
    MAverage10  MAverage20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
58  184.792999  186.889499
59  184.212999  186.306999
60  183.443999  185.743999
61  182.992000  185.289000
62  182.604970  184.945485

[63 rows x 2 columns]



Moving Average for GOOG
     MAverage10  MAverage20
189         NaN         NaN
190         NaN         NaN
191         NaN         NaN
192         NaN         NaN
193         NaN         NaN
..          ...         ...
247  145.673999  146.856499
248  145.480998  146.438998
249  144.333998  145.686999
250  143.470999  144.949999
251  142.593500  144.177249

[63 rows x 2 columns]



Moving Average for IBM
     MAverage10  MAverage20
252         NaN         NaN
253         NaN         NaN
254         NaN         NaN
255         NaN         NaN
256         NaN         NaN
..          ...         ...
310  184.599001  185.

In [10]:
for ticker,group in df.groupby('Ticker'):
    img_3 = plexp.line(
        group,
        x= 'Date',
        y= ['Close', 'MAverage10', 'MAverage20'],
        title= f"Stock Price and Moving Averages for {ticker}",
    )
    img_3.show()

In [11]:
df['volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
img_4 = plexp.line(
    df,x= 'Date',
    y= 'volatility',
    color= 'Ticker',
    title= 'Volatility of Stocks'
)
img_4.show()


In [12]:
apple = df.loc[df['Ticker'] == 'AAPL',['Date','Close']].rename(columns={'Close':'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT',['Date','Close']].rename(columns={'Close':'MSFT'})
correlation = pa.merge(apple,microsoft,on='Date')
img_5 = plexp.scatter(correlation,
                      x = 'AAPL', y = "MSFT",
                      trendline= 'expanding',
                      title="Correlation between APPLE and MICROSOFT Stocks")
img_5.show()